In [37]:
from pathlib import Path

import pandas as pd
from typing import Tuple, Union

import torch
import torchaudio

from torch.utils.data import Dataset

In [43]:
_RELEASE_CONFIGS = {
    'release1': {
        "folder_in_archive": "the-circor-digiscope-phonocardiogram-dataset-1.0.3/training_data",
        "url": "https://physionet.org/static/published-projects/circor-heart-sound/the-circor-digiscope-phonocardiogram-dataset-1.0.3.zip"
    }
}

class Phonocardiogram(Dataset):
    def __init__(
        self,
        root: Union[str, Path],
        url: str = _RELEASE_CONFIGS['release1']['url'],
        folder_in_archive: str = _RELEASE_CONFIGS['release1']['folder_in_archive'],
        download: bool = False,
    ) -> None:
        self._root = Path(root)
        self._dataset_dir = self._root / folder_in_archive
        self._patient_ids = [p.stem for p in self._dataset_dir.glob('*.wav')]
    
    def __getitem__(self, n: int) -> Tuple[torch.Tensor, int, torch.Tensor, torch.Tensor]:
        patient_id = self._patient_ids[n]
        wav_path = self._dataset_dir / (patient_id + '.wav')
        tsv_path = self._dataset_dir / (patient_id + '.tsv')
        
        waveform, sample_rate = torchaudio.load(wav_path)
        
        labels_df = pd.read_csv(tsv_path, delimiter='\t', header=None)
        s1_ranges = torch.tensor(labels_df[labels_df[2] == 1].drop(2, axis=1).values)
        s2_ranges = torch.tensor(labels_df[labels_df[2] == 3].drop(2, axis=1).values)
        
        return waveform, sample_rate, s1_ranges, s2_ranges
    
    def __len__(self) -> int:
        return len(self._patient_ids)

In [44]:
dataset = Phonocardiogram(root='dataset', folder_in_archive='training_data')

len(dataset)

3163

In [45]:
waveform, sample_rate, s1, s2 = dataset[1]

In [46]:
print(waveform.shape, s1.shape, s2.shape)

torch.Size([1, 79552]) torch.Size([27, 2]) torch.Size([26, 2])


In [47]:
waveform, sample_rate, s1, s2

(tensor([[-0.0272,  0.0637,  0.0013,  ...,  0.0000,  0.0040,  0.0138]]),
 4000,
 tensor([[ 0.1052,  0.2602],
         [ 0.6602,  0.7802],
         [ 1.2002,  1.3202],
         [ 1.7202,  1.8202],
         [ 2.2402,  2.3602],
         [ 2.7402,  2.8602],
         [ 3.2602,  3.3802],
         [ 3.7202,  3.8602],
         [ 4.2402,  4.3602],
         [ 4.7202,  4.8202],
         [ 5.2002,  5.3202],
         [ 5.7202,  5.8202],
         [ 6.2202,  6.3402],
         [ 6.7202,  6.8602],
         [ 7.2402,  7.3802],
         [ 7.7802,  7.8802],
         [ 8.3002,  8.4202],
         [ 8.8202,  8.9402],
         [ 9.3602,  9.4602],
         [ 9.8202, 10.0002],
         [10.4202, 10.5802],
         [10.9802, 11.0802],
         [11.5002, 11.6202],
         [12.0202, 12.1402],
         [12.5402, 12.6602],
         [13.0402, 13.1602],
         [13.5202, 13.6402]], dtype=torch.float64),
 tensor([[ 0.3602,  0.4802],
         [ 0.8802,  1.0002],
         [ 1.4202,  1.5402],
         [ 1.9202,  2.0602]